In [1]:
    import numpy as np
    from sklearn.model_selection import train_test_split

    data = np.load('olivetti_faces.npy')
    target = np.load('olivetti_faces_target.npy')

    classes = 41
    noise_size = 100
    width = 64
    height = 64
    channel = 1

    data = data.reshape(data.shape[0],data.shape[1],data.shape[2],channel)
    
#     X_train, X_test, y_train, y_test = train_test_split(data, target,test_size=0, shuffle=True)
    
    X_train = data
    y_train = target
    
    import Discriminator
    from keras.layers import Dense,Input
    from keras.models import Model
    from keras.optimizers import Adam, Nadam ,Adamax , SGD
    
    Img = Input(shape=(width,height,channel))
    x = Discriminator.paper_D(Img)
    x1 = Dense(classes,activation='softmax',name='label')(x)
    
    D = Model(name='Paper_Discriminator' ,inputs= Img, outputs=x1)
    
    from keras.optimizers import SGD
    D.trainable = True
    D.compile(  loss= ['sparse_categorical_crossentropy'],
                optimizer= Adam(lr=0.0002 ,decay=1e-8) ,
                metrics=['accuracy'])
    
    import Generator
    from keras.layers import Dense,Input
    from keras.models import Model
    
    inpt1 = Input(shape=(noise_size,))
    x = Generator.paper_G(inpt1)

    G = Model(name='Paper_Generator', inputs=[inpt1], outputs=x)
    
    
    from keras.layers import Dense, Activation, Dropout, Conv2D, Input, Reshape, UpSampling2D ,concatenate ,BatchNormalization, Embedding, MaxPooling2D,Conv2DTranspose ,Flatten, multiply
    from keras.models import Model
    from keras.models import Sequential, Model
    from keras.optimizers import Adam, Nadam ,Adamax , SGD

    D.trainable = False
    inpt1 = Input(shape=(noise_size,))
    img = G([inpt1])
    classes = D(img)
    C =  Model(name='Combined', inputs=[inpt1], outputs=[classes])

    C.compile(  loss= ['sparse_categorical_crossentropy'],
                optimizer= Adam(lr=0.0002 ,decay=1e-8),
                metrics=['accuracy']
                )
    

    import matplotlib.pyplot as plt
    import os
    count = 0
    atk = 1 # attack number
    sz = 15
    gs = 400
    dik = 'normal[{}]({}_{})'.format(atk,gs,sz)
    
    try :
        os.mkdir('./{}'.format(dik))
    except:
        None
        
    label = np.random.randint(low=atk, high= atk+1, size=(gs, 1))
    fake = np.random.randint(low=40, high= 41, size=(sz, 1))
    while (count<400):
        bs = 32 # batch_szie
        count += 1

        print ('{}_{} | Count:{}/400'.format(gs,sz,count))

        # round 1 (400)
        D_history = D.fit(
                x=[X_train], y=[y_train] ,validation_split=0, epochs= 1, batch_size=bs, verbose=1)

        # round 2 (400)
        noise = np.random.normal(loc=0, scale=1, size=(gs, noise_size))


        G_history = C.fit(
                x=[noise], y=[label] ,validation_split=0, epochs= 1, batch_size= bs, verbose=1)

        # round 3 (10)
        noise = np.random.normal(loc=0, scale=1, size=(sz, noise_size))
        img = G.predict(noise)
        score = D.fit(
                x=[img], y=[fake] ,validation_split=0, epochs= 1, batch_size=bs, verbose=1)
        
        draw = np.reshape(img[0],(64,64))
        plt.imsave('./{}/Generator[{}]{}.png'.format(dik,atk,str(count).zfill(6)), draw, cmap='gray')


Using TensorFlow backend.


400_15 | Count:1/400


/home/adi/miniconda3/envs/py3.7/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/1
400/400 [==============================] - 2s 5ms/step - loss: 3.7567 - accuracy: 0.0075
Epoch 1/1
400/400 [==============================] - 1s 3ms/step - loss: 3.7654 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 2ms/step - loss: 3.7317 - accuracy: 0.0000e+00
400_15 | Count:2/400
Epoch 1/1
400/400 [==============================] - 0s 304us/step - loss: 3.6742 - accuracy: 0.0325


/home/adi/miniconda3/envs/py3.7/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

400/400 [==============================] - 0s 314us/step - loss: 0.0252 - accuracy: 0.9850
Epoch 1/1
400/400 [==============================] - 0s 592us/step - loss: 2.0333 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 658us/step - loss: 0.1884 - accuracy: 1.0000
400_15 | Count:278/400
Epoch 1/1
400/400 [==============================] - 0s 313us/step - loss: 0.0255 - accuracy: 0.9825
Epoch 1/1
400/400 [==============================] - 0s 607us/step - loss: 1.6396 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 669us/step - loss: 0.2912 - accuracy: 1.0000
400_15 | Count:279/400
Epoch 1/1
400/400 [==============================] - 0s 329us/step - loss: 0.0341 - accuracy: 0.9825
Epoch 1/1
400/400 [==============================] - 0s 601us/step - loss: 2.4399 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 662us/step - loss: 0.1336 - accuracy: 1.0000
400_15 | Count:280/400
Epoch 1/1
400/400 [=======

400/400 [==============================] - 0s 327us/step - loss: 0.0408 - accuracy: 0.9775
Epoch 1/1
400/400 [==============================] - 0s 603us/step - loss: 2.6736 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 645us/step - loss: 0.1157 - accuracy: 1.0000
400_15 | Count:303/400
Epoch 1/1
400/400 [==============================] - 0s 316us/step - loss: 0.0281 - accuracy: 0.9850
Epoch 1/1
400/400 [==============================] - 0s 601us/step - loss: 1.4713 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 661us/step - loss: 0.4414 - accuracy: 1.0000
400_15 | Count:304/400
Epoch 1/1
400/400 [==============================] - 0s 301us/step - loss: 0.0541 - accuracy: 0.9775
Epoch 1/1
400/400 [==============================] - 0s 602us/step - loss: 3.9153 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 638us/step - loss: 0.0387 - accuracy: 1.0000
400_15 | Count:305/400
Epoch 1/1
400/400 [=======

400/400 [==============================] - 0s 308us/step - loss: 0.0195 - accuracy: 0.9925
Epoch 1/1
400/400 [==============================] - 0s 601us/step - loss: 1.9478 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 673us/step - loss: 0.2759 - accuracy: 1.0000
400_15 | Count:328/400
Epoch 1/1
400/400 [==============================] - 0s 333us/step - loss: 0.0425 - accuracy: 0.9800
Epoch 1/1
400/400 [==============================] - 0s 597us/step - loss: 3.4584 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 665us/step - loss: 0.0653 - accuracy: 1.0000
400_15 | Count:329/400
Epoch 1/1
400/400 [==============================] - 0s 321us/step - loss: 0.0257 - accuracy: 0.9900
Epoch 1/1
400/400 [==============================] - 0s 597us/step - loss: 0.9703 - accuracy: 0.3625
Epoch 1/1
15/15 [==============================] - 0s 687us/step - loss: 1.2138 - accuracy: 0.0000e+00
400_15 | Count:330/400
Epoch 1/1
400/400 [=======

400_15 | Count:352/400
Epoch 1/1
400/400 [==============================] - 0s 316us/step - loss: 0.0366 - accuracy: 0.9825
Epoch 1/1
400/400 [==============================] - 0s 594us/step - loss: 1.5769 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 679us/step - loss: 0.3347 - accuracy: 1.0000
400_15 | Count:353/400
Epoch 1/1
400/400 [==============================] - 0s 328us/step - loss: 0.0452 - accuracy: 0.9800
Epoch 1/1
400/400 [==============================] - 0s 601us/step - loss: 3.2484 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 643us/step - loss: 0.0715 - accuracy: 1.0000
400_15 | Count:354/400
Epoch 1/1
400/400 [==============================] - 0s 327us/step - loss: 0.0278 - accuracy: 0.9800
Epoch 1/1
400/400 [==============================] - 0s 597us/step - loss: 0.8273 - accuracy: 0.4900
Epoch 1/1
15/15 [==============================] - 0s 649us/step - loss: 1.6476 - accuracy: 0.0000e+00
400_15 | Count:3

400_15 | Count:377/400
Epoch 1/1
400/400 [==============================] - 0s 316us/step - loss: 0.0252 - accuracy: 0.9900
Epoch 1/1
400/400 [==============================] - 0s 604us/step - loss: 1.7165 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 671us/step - loss: 0.3485 - accuracy: 1.0000
400_15 | Count:378/400
Epoch 1/1
400/400 [==============================] - 0s 313us/step - loss: 0.0309 - accuracy: 0.9875
Epoch 1/1
400/400 [==============================] - 0s 590us/step - loss: 3.3622 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 623us/step - loss: 0.0664 - accuracy: 1.0000
400_15 | Count:379/400
Epoch 1/1
400/400 [==============================] - 0s 317us/step - loss: 0.0269 - accuracy: 0.9875
Epoch 1/1
400/400 [==============================] - 0s 606us/step - loss: 1.6854 - accuracy: 0.0000e+00
Epoch 1/1
15/15 [==============================] - 0s 673us/step - loss: 0.4305 - accuracy: 1.0000
400_15 | Count:3